In [ ]:
import pandas as pd
import os
import pathlib

currentPath = pathlib.Path().resolve()
print(currentPath)

In [ ]:
dir_path = os.path.abspath(os.path.join(currentPath,"data"))

fileName = "Edison's Scores.xlsx"

fileLocation = os.path.join(dir_path, fileName)

df = pd.read_excel(fileLocation)
print(df.head(3))

In [ ]:
from typing import Optional

class HoleResult():
    def __init__(self, hole: str, input: str) -> None:
        holeResults : list[str] = input.split(",")
        self.hole : str = hole
        
        par = holeResults[0]
        if par[0] == "'":
            par = par.replace(par[0], "", 1)
            
        self.par : float = float(par)
        self.distance: float = float(holeResults[1])
        self.toGreen: float = float(holeResults[2])
        self.putts: float = float(holeResults[3])
        self.total: float = float(holeResults[4])
        
        self.fir : Optional[float] = None
        if len(holeResults) > 5:
            fir = holeResults[5]
            
            if fir == "X":
                self.fir = 0.0
            else:
                self.fir = 1.0
        
class RoundResult():
    def __init__(self, location: str, division: str, date : str) -> None:
        self.location : str = location
        self.division : str = division
        self.date : str = date
        self.holes : dict[HoleResult] = {}
        
    def addHole(self, holeResult : HoleResult):
        self.holes[holeResult.hole] = holeResult

In [ ]:
holes = ["Hole 1",	"Hole 2",	"Hole 3",	 "Hole 4",	"Hole 5",	"Hole 6",	"Hole 7",	"Hole 8",	"Hole 9"]


In [ ]:
rounds:list[RoundResult] = []

#df = df.reset_index()  # make sure indexes pair with number of rows

for index, row in df.iterrows():
    round = RoundResult(location = row['Course'], division=row['Division'], date=row['Date'])
    
        
    for holeName in holes:
        holeResult = HoleResult(hole=holeName, input=row[holeName])
        round.addHole(holeResult=holeResult)
    
    rounds.append(round)

In [ ]:
# Average Putts per Three Holes
dates_threehole = []
dates = []
puttAverage = []
totalScore = []

avgPar3List = []
avgPar4List = []
avgPar5List = []

girList = []


for round in rounds:
    my_dict : dict[HoleResult] = round.holes
    
    
    # Average per par type
    fir = 0
    gir = 0
    scrambleChance = 0
    scrambleSuccess = 0
    
    numberPar3 = 0
    totalPar3 = 0
    numberPar4 = 0
    totalPar4 = 0
    numberPar5 = 0
    totalPar5 = 0
    
    for hole in holes:
        
        if hole.fir is not None:
            fir += hole.fir
        
        if hole.par == 3:
            numberPar3+=1
            if hole.toGreen == 1:
                gir+=1
            else:
                scrambleChance+=1
                
                if hole.putts == 1:
                    scrambleSuccess+=1
                
            totalPar3 += hole.total
            
        elif hole.par == 4:
            numberPar4 += 1
            if hole.toGreen == 2:
                gir+=1
            else:
                scrambleChance+=1
                
                if hole.putts == 1:
                    scrambleSuccess+=1
            
            totalPar4 += hole.total
            
        else:
            numberPar5 += 1
            if hole.toGreen == 3:
                gir+=1
            else:
                scrambleChance+=1
                
                if hole.putts == 1:
                    scrambleSuccess+=1
                
            totalPar5 += hole.total
    
    gir = gir/9
    fir = fir/9
    
    girList.append(gir)
    
    avgPar3 = totalPar3/numberPar3
    avgPar4 = totalPar4/numberPar4
    avgPar5 = totalPar5/numberPar5
    
    avgPar3List.append(avgPar3)
    avgPar4List.append(avgPar4)
    avgPar5List.append(avgPar5)
    
    ## ============================================
    score = 0
    
    numberOfPutts = 0
    for hole in holes[0:3]:
        numberOfPutts += my_dict[hole].putts/3.0
        score += my_dict[hole].total
    
    puttAverage.append(numberOfPutts)
    dates_threehole.append(round.date)
    
    numberOfPutts = 0
    for hole in holes[3:6]:
        numberOfPutts += my_dict[hole].putts/3.0
        score += my_dict[hole].total
    
    puttAverage.append(numberOfPutts)
    dates_threehole.append(round.date)
    
    numberOfPutts = 0
    for hole in holes[6:9]:   
        numberOfPutts += my_dict[hole].putts/3.0
        score += my_dict[hole].total
    
    totalScore.append(score)
    dates.append(round.date)
    
    puttAverage.append(numberOfPutts)
    dates_threehole.append(round.date)
    
dfGirAverage =  pd.DataFrame(list(zip(dates, girList)),
               columns =['Dates', 'GIR'])   

dfHoleAverage = pd.DataFrame(list(zip(dates, avgPar3List, avgPar4List, avgPar5List)),
               columns =['Dates', 'Par 3', 'Par 4', 'Par 5'])    

dfPutts = pd.DataFrame(list(zip(dates_threehole, puttAverage)),
               columns =['Dates', 'Putts'])

dfScore = pd.DataFrame(list(zip(dates, totalScore)),
               columns =['Dates', 'Score'])

dfGirAverage['Dates'] = pd.to_datetime(dfGirAverage['Dates'])
dfGirAverage = dfGirAverage.set_index('Dates')

dfHoleAverage['Dates'] = pd.to_datetime(dfHoleAverage['Dates'])
dfHoleAverage = dfHoleAverage.set_index('Dates')

dfScore['Dates'] = pd.to_datetime(dfScore['Dates'])
dfScore = dfScore.set_index('Dates')

dfPutts['Dates'] = pd.to_datetime(dfPutts['Dates'])
dfPutts = dfPutts.set_index('Dates')

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime

In [ ]:
fig,ax =plt.subplots()
plt.xticks(rotation=70)
plt.xlabel("Date")
plt.ylabel("Mean Putts over Three Holes")
plt.grid(True)
ax.scatter(dfPutts.index, dfPutts['Putts'])

In [ ]:
fig,ax =plt.subplots()
plt.xticks(rotation=70)
plt.xlabel("Date")
plt.ylabel("Total Score")
plt.grid(True)
ax.scatter(dfScore.index, dfScore['Score'])